In [157]:
import requests
from bs4 import BeautifulSoup

from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import json
from pprint import pprint
from tqdm import tqdm
import pandas as pd

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/85/BlackRock_wordmark.svg/2560px-BlackRock_wordmark.svg.png" width=350>

In [98]:
class BlackRockInsights:
    
    def __init__(self, all_insight_link):
        self.all_insight_link = all_insight_link
        self.driver = webdriver.Chrome()
    
    @staticmethod
    def extract_content_with_p(link: str) -> str:
        # send requests and get response
        response = requests.get(link)

        # get content if response is ok
        if response.ok is True:
            soup = BeautifulSoup(response.text)

            content = " ".join([i.text for i in soup.find_all('p')[5:]])
            return content
        else:
            raise Exception(f'Response is: {response}')
            
    @staticmethod        
    def extract_content_with_class(link: str) -> str:
        #send requests 
        response = requests.get(link)
        soup = BeautifulSoup(response.text)

        content = ' '.join([i.text for i in soup.find_all(class_='para-content')])
        return content
            
    def get_article_insight(self):
        
        self.driver.get(self.all_insight_link)
        sleep(10)
        elem = self.driver.find_element(By.ID,'onetrust-reject-all-handler')
        elem.click()
        button = self.driver.find_element(By.XPATH,'//*[@id="c1561158932259"]/div/div[2]/div/div/div[3]/button')
        for i in range(8):
            self.driver.execute_script("arguments[0].click();", button)

        title_list = [i.text for i in self.driver.find_elements(By.XPATH,'//h2')]
        date_list = [i.text for i in self.driver.find_elements(By.XPATH, '//a//div//div//span[(((count(preceding-sibling::*) + 1) = 1) and parent::*)]')]
        author_list = [i.text for i in self.driver.find_elements(By.CSS_SELECTOR,'span:nth-child(4)')] 
        abstract_list = [i.text for i in self.driver.find_elements(By.XPATH,'//*[contains(concat( " ", @class, " " ), concat( " ", "description", " " ))]')]
        link_list = [i.get_attribute('href') for i in self.driver.find_elements(By.CSS_SELECTOR,'.skip-animation')]

        blackrock_insight_list = []
        for title,date,author,abstract,link in tqdm(zip(title_list,date_list,author_list,abstract_list,link_list)):
            
            blackrock_insight_dict = {
                'company': 'BlackRock', 
                'topic': 'Insight',
                'title':title,
                'link':link,
                'date':date,
                'author':author,
                'abstract':abstract,
                'content': self.extract_content(link),
            }
            
            blackrock_insight_list.append(blackrock_insight_dict)
        
        return blackrock_insight_list
    

    @staticmethod
    def save_to_json(file_path, list_of_dict):
        with open(file_path, 'a') as f:
            for dict_ in list_of_dict:
                json.dump(dict_,f,indent=4)



In [99]:
obj = BlackRockInsights('https://www.blackrock.com/us/individual/insights')
l = obj.get_article_insight()

50it [00:51,  1.03s/it]


<img src="https://1000logos.net/wp-content/uploads/2021/05/Vanguard-logo.png" width=350 style="float: center;">

In [102]:
requests.get('https://advisors.vanguard.com/insights/article/vanguardtointroduceenvironmentalopportunitiesfund').text

'<!DOCTYPE html>\r\n<html lang="en">\n<head>\n<META http-equiv="Content-Type" content="text/html; charset=UTF-8">\n<title>Vanguard to introduce Environmental Opportunities Fund</title>\n<!--grid-layout-->\n<!--ls:begin[stylesheet]-->\n<style type="text/css">\n          \n          .iw_container\n          {\n            max-width:800px !important;\n            margin-left: auto !important;\n            margin-right: auto !important;\n          }\n          .iw_stretch\n          {\n            min-width: 100% !important;\n          }\n        </style>\n<link href="/iwov-resources/grid/bootstrap/4.0/bootstrap-grid.min.css" type="text/css" rel="stylesheet">\n<!--ls:end[stylesheet]-->\n<!--ls:begin[canonical]-->\n<!--ls:end[canonical]-->\n<!--ls:begin[script]-->\n<!--ls:end[script]-->\n<!--ls:begin[meta-keywords]-->\n<meta name="keywords" content="">\n<!--ls:end[meta-keywords]-->\n<!--ls:begin[meta-description]-->\n<meta name="description" content="Vanguard announces its intention to laun

In [103]:
def extract_content(link: str) -> str:
        # send requests and get response
        response = requests.get(link)

        # get content if response is ok
        if response.ok is True:
            soup = BeautifulSoup(response.text)

            content = " ".join([i.text for i in soup.find_all('p')])
            return content
        else:
            raise Exception(f'Response is: {response}')

In [156]:
extract_content('https://www.blackrock.com/us/individual/insights/systematic-equity-market-outlook')

"  I invest on behalf of my clients. I consult or invest on behalf of a financial institution. I want to learn more about BlackRock. To reach a different BlackRock site directly, please update your user type.\xa0 Macro volatility driving equity markets  Markets have entered a new regime characterized by higher levels of volatility and uncertainty as central banks try to rein in inflation. Will the inflation fight cause a recession?  As recession fears rise, alternative data indicators reveal that markets may not be accurately pricing recession risk despite inflationary pressures. Positioning for a new market regime  Slowing growth and a new macro regime may require rethinking portfolio construction and adopting a more dynamic investment approach. Since the Great Financial Crisis (“GFC”), we’ve been in an environment characterized by strong financial market returns and low volatility against the backdrop of ultra-accommodative monetary and fiscal policy. A continuous stream of upside su

In [105]:
extract_content('https://advisors.vanguard.com/insights/article/offerclientscustomesgoptionswithpersonalizedindexing')

'August 4, 2022 | Vanguard Perspective Your clients want to make sure their investments reflect the issues they care about and avoid supporting others. Their desire has made environmental, social, and governance (ESG) investing, also known as socially responsible, faith-based, values-driven, or sustainable investing, a burgeoning trend.1 It\'s true that you can help align clients\' investments with their preferences using off-the-shelf ETFs and mutual funds. But for clients with very specific ideas about causes or issues they want their investments to support or exclude, Vanguard Personalized Indexing could be a more precise solution. Like many people, your clients may focus mostly on the "E" of ESG—that is, environmental concerns like a company\'s carbon footprint or involvement with fossil fuels—but ESG investing can cover a much wider range of issues. For example, your clients can: What makes a personalized indexing portfolio so flexible? It\'s all in the structure—and its easy-to-u

In [138]:
driver = webdriver.Chrome()
driver.get('https://www.capitalgroup.com/advisor/insights.html')
sleep(15)
buttton = driver.find_element(By.CSS_SELECTOR,'#blue-box > div > div.text.cg-global-body-2.cg-padding-bottom--mobile--20.aem-GridColumn--default--none.aem-GridColumn--phone--none.aem-GridColumn--phone--12.aem-GridColumn.aem-GridColumn--default--10.aem-GridColumn--offset--phone--0.aem-GridColumn--offset--default--1 > div > p > button')
buttton.click()
link_list = [i.get_attribute('href') for i in driver.find_elements(By.CSS_SELECTOR,'article > a')]








In [ ]:
#blue-box > div > div.text.cg-global-body-2.cg-padding-bottom--mobile--20.aem-GridColumn--default--none.aem-GridColumn--phone--none.aem-GridColumn--phone--12.aem-GridColumn.aem-GridColumn--default--10.aem-GridColumn--offset--phone--0.aem-GridColumn--offset--default--1 > div > p > button

In [127]:
len(articles_a)

30

In [139]:
len(link_list)

30

In [140]:
link_list

['https://www.capitalgroup.com/advisor/insights/webinars/replay-all-weather-investing-amid-storm.html',
 'https://www.capitalgroup.com/advisor/insights/articles/pandemic-war-and-climate-change-sharpen-focus-on-esg.html',
 'https://www.capitalgroup.com/advisor/insights/articles/3-ways-prepare-down-markets.html',
 'https://www.capitalgroup.com/advisor/insights/webinars/2022-midyear-outlook.html',
 'https://www.capitalgroup.com/advisor/insights/articles/fed-aggressive-path-could-mean-more-market-turmoil.html',
 'https://www.capitalgroup.com/advisor/insights/articles/2022-midyear-bond-outlook.html',
 'https://www.capitalgroup.com/advisor/insights/articles/2022-midyear-us-outlook.html',
 'https://www.capitalgroup.com/advisor/insights/articles/2022-midyear-international-outlook.html',
 'https://www.capitalgroup.com/advisor/insights/articles/labor-inflation-charts.html',
 'https://www.capitalgroup.com/advisor/insights/articles/investing-in-china-today.html',
 'https://www.capitalgroup.com/adv

In [ ]:
body > div.root.responsivegrid.cg-wt-global-styles.cg-full-bleed > div > div.articlelist.list.cmp-articlelist-insights-default.cg-padding-bottom--desktop--80.aem-GridColumn.aem-GridColumn--default--12 > ul > li:nth-child(1) > article > a